[Home](http://realai.org/) > [Course](http://realai.org/course/) > [TensorFlow](http://realai.org/course/tensorflow/) > [Classifying Handwritten Digits](http://realai.org/course/tensorflow/#classifying-handwritten-digits) >

# MNIST Softmax Regression

*Last Updated: September 18, 2017*

The MNIST data downloaded from the [last session](http://realai.org/course/tensorflow/#setting-up-your-computer) are reusable. The next cell should only print "Extracting ...", without any line that starts with "Successfully downloaded".

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


We already know the shape of the data. Our goal is to recognize the digits in images. Initially these images are a batch of 784-dimensional vectors in

In [2]:
images = tf.placeholder(tf.float32, (None, 784), name="Images")

Our goal is to classify images based on the digits they contain. Take one "image" of numbers, $image_0, image_1, ..., image_{783}$, and let's focus on a simpler problem, decide whether the numbers represent the digit 0. One of the simplest models is [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression). The basic idea is to find "weights" $w_0, w_1, ..., w_{783}$ and a bias $b$ such that

$t = b + image_0 w_0 + image_1 w_1 + ... + image_{783} w_{783}$

is a good indicator of the "zeroness" of the image. Bigger $t$ value indicates higher confidence that the image is 0.

Softmax regression is just a generalization of this to multiple classes, one linear relation for each class. We add a new subscript to the above formula to indicate the digit class, then repeat:

$t_0 = b_0 + image_0 w_{0, 0} + image_1 w_{1, 0} + ... + image_{783} w_{783, 0}$

$t_1 = b_1 + image_0 w_{0, 1} + image_1 w_{1, 1} + ... + image_{783} w_{783, 1}$

...

$t_9 = b_9 + image_0 w_{0, 9} + image_1 w_{1, 9} + ... + image_{783} w_{783, 9}$

These match exactly with the formula of [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication), and in TensorFlow it takes only three lines of code:

In [3]:
W = tf.Variable(tf.truncated_normal((784, 10), mean=0.1, stddev=0.1), name="Weights")
with tf.name_scope("Biases") as scope:
    B = tf.Variable(tf.ones((1, 10)))
logits = tf.add(tf.matmul(images, W), B, name="Logits")

In the above cell, `W` and `B` belong to the class [`tf.Variable`](https://www.tensorflow.org/api_docs/python/tf/Variable). Unlike [`tf.placeholder`](https://www.tensorflow.org/api_docs/python/tf/placeholder), the values held in variables persist across multiple calls to `run()` in a session. They commonly represent the parameters of a model that encode the model's knowledge about input and output data. The model's performance gradually improves as these values change during training. The initial biases are equal across all digits, and their value is typically much greater than the values in `images` and `W`. They're deliberately chosen so that the model starts out predicting roughly equal probabilities among all digits

This transformation is an example of a layer in a neural network. Conceptually, the `images` and `logits` tensors are analogous to neurons, and the weights and biases analogous to connections between them. In our scenario the layer is said to be *fully-connected*, or *dense* in TensorFlow terminology, since there is a weight connecting every pair of images-logits neurons. Fully-connected layers are widely used in practice. TensorFlow has a higher level interface [tf.layers.dense](https://www.tensorflow.org/api_docs/python/tf/layers/dense) that implements a more sophisticated version of the above, which allows us to simply use one line

`logits = tf.layers.dense(images, 10, activation=None, name="Logits")`

instead of the three lines containing variables `W` and `B`. The rest of this script works with both versions. At this point `logits` should be a (None, 10)-shaped tensor, representing the results of 10 separate linear regressions, one for each digit class:

In [4]:
logits.shape

TensorShape([Dimension(None), Dimension(10)])

To complete the [softmax regression](http://ufldl.stanford.edu/tutorial/supervised/SoftmaxRegression/), we will need to turn the logits into probabilities for each class. Then we match them with the labels in the data set to measure our model's performance. Given logits $z_0$, $z_1$, ..., $z_9$, which can be any real numbers, the [softmax function](https://en.wikipedia.org/wiki/Softmax_function) is a natural formula that turns them into probabilities:

$$s_j = \frac{e^{z_j}}{e^{z_0} + e^{z_1} + ... + e^{z_9}}$$

To measure how these probabilities match with true labels, we use the [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) between the one-hot labels and the softmax values:

$$L = y_0 log(s_0) + ... + y_9 log(s_9)$$

Because the labels are one-hot, one image with label $l$ simply contributes one term, $log(s_l)$, to the loss $L$. For each batch of images, the total loss for that batch is the average of the losses from all images.

These formulas involve quite a bit of mathematics and may look daunting. But fortunately TensorFlow has done the heavy lifting for its users. Softmax and cross-entropy loss are conveniently implemented together in [tf.nn.softmax_cross_entropy_with_logits](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits). In the TensorFlow world, we only need to follow normal softmax regression procedure to the step of computing logits, then we can skip to the loss function in just one step of calculation:

In [5]:
labels = tf.placeholder(tf.float32, (None, 10), name="Labels")

with tf.name_scope("Loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits),
        name="Mean")

The loss function above measures how well our model predictions match the true labels. At this point, we've set up the digit classification problem as a standard optimization problem. We'd like to find the appropriate parameters `W` and `B`, or whatever parameters implicitly defined in `tf.layers.dense` if we take the shortcut, that make the value of "loss" as low as it can be. In deep learning, this is commonly achieved by [gradient descent](http://realai.org/course/deep-learning/gradient-descent/), the common name of a variety of weight update rules that hopefully achieve lower and lower loss values. In its most basic form, we simply update the parameters $w_i$ by a small amount

$$\Delta w_i = -\alpha \frac{\partial L}{\partial w_i}$$

per training step, where $\alpha$ is the learning rate, and we abuse the notation $L$ to also let it denote the loss value for each batch of input data. In practice, several more complex rules have been developed for good reasons. Again, we don't have to implement the mathematical details, there is a TensorFlow class for that called [Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer), with a range of popular gradient descent variants such as [tf.train.GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer), [tf.train.AdagradOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/AdagradOptimizer) and [tf.train.AdadeltaOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/AdadeltaOptimizer). These optmizers should all work well for our simple case of regression, so in this session we simply pick a popular variant, [tf.train.AdamOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer), with a reasonably small learning rate of 0.001. The learning rate affects how much the model parameters change in each optimization step. A learning rate too big might cause the parameters to jump around optimal values and never reach there, while the cost of a small learning rate is slower convergence. In this relatively simple setup, reasonably small learning rates such as 0.001 or 0.0001 work equally well:

In [6]:
with tf.name_scope("Optimizer"):
    train = tf.train.AdamOptimizer(learning_rate=0.001, name="Adam").minimize(loss)

The target of our optimization is the loss function, but it is rather unintuitive measure of how well our model performs. Since the model's goal is classification, we'd like to measure its error rate. In `logits`, the model outputs a "logit" for each class instead of a single class prediction. We can pick the largest logit to be the model's prediction, and see if it matches with the label:

In [7]:
with tf.name_scope("Error"):
    error = tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(labels, 1), tf.argmax(logits, 1)), tf.float32), name="Mean")

Add a couple summary nodes so that we can easily watch the training progress in TensorBoard:

In [8]:
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Error", error)
summ = tf.summary.merge_all()

The computation graph is now fully set up and should look like

![](http://realai.org/course/tensorflow/MNIST-softmax-regression-1.png)

In [9]:
LOGDIR = "/tmp/MNIST_softmax"
if tf.gfile.Exists(LOGDIR):
    tf.gfile.DeleteRecursively(LOGDIR)
tf.gfile.MakeDirs(LOGDIR)

Start session and writer

In [10]:
sess = tf.Session()
writer = tf.summary.FileWriter(LOGDIR, sess.graph)

Don't forget to initialize variables, even if you use `tf.layers.dense`, in which case the variables are not declared explicitly in the code, but still exist. They're just wrapped inside the dense layer:

In [11]:
sess.run(tf.global_variables_initializer())

In [12]:
for i in range(10000):
    batch = MNIST.train.next_batch(64)
    
    if i % 10 == 0:
        Error, Loss, Summ = sess.run((error, loss, summ), feed_dict={images: batch[0], labels: batch[1]})
        writer.add_summary(Summ, i)
        
        if i % 500 == 0:
            print("Step {}: Training loss is {:.5f}, error is {:.2f}%".format(i, Loss, Error * 100))

    sess.run(train, feed_dict={images: batch[0], labels: batch[1]})

Step 0: Training loss is 2.40330, error is 84.38%
Step 500: Training loss is 0.28341, error is 6.25%
Step 1000: Training loss is 0.31295, error is 7.81%
Step 1500: Training loss is 0.21901, error is 6.25%
Step 2000: Training loss is 0.37929, error is 14.06%
Step 2500: Training loss is 0.33128, error is 12.50%
Step 3000: Training loss is 0.21838, error is 9.38%
Step 3500: Training loss is 0.55060, error is 12.50%
Step 4000: Training loss is 0.12710, error is 3.12%
Step 4500: Training loss is 0.17055, error is 3.12%
Step 5000: Training loss is 0.30132, error is 6.25%
Step 5500: Training loss is 0.35612, error is 7.81%
Step 6000: Training loss is 0.24885, error is 7.81%
Step 6500: Training loss is 0.30968, error is 6.25%
Step 7000: Training loss is 0.16556, error is 4.69%
Step 7500: Training loss is 0.09393, error is 0.00%
Step 8000: Training loss is 0.21135, error is 7.81%
Step 8500: Training loss is 0.34668, error is 10.94%
Step 9000: Training loss is 0.25505, error is 4.69%
Step 9500: 

In [13]:
Error = sess.run(error, feed_dict={images: MNIST.validation.images, labels: MNIST.validation.labels})
print("Validation error is {:.2f}%".format(Error * 100))

Validation error is 7.06%


Close FileWriter and Session:

In [14]:
writer.close()
sess.close()